In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

from config import weather_api_key

In [4]:
data = 'winery_latlong_data.csv'
latlong_df = pd.read_csv(data, low_memory=False)

In [5]:
latlong_df.dropna(inplace=True)
latlong_df

,Unnamed: 0,winery_search_term,latitude,longitude,number_of_results,status
0,0,St. Julian winery US,42.212251,-85.891713,1,OK
1,1,Sweet Cheeks winery US,43.956670,-123.279135,1,OK
2,2,Kirkland Signature winery US,35.896713,-84.140546,1,OK
3,3,Louis M. Martini winery US,38.491708,-122.452308,1,OK
5,5,Richard Böcking winery Germany,49.947177,7.112136,1,OK
...,...,...,...,...,...,...
8693,8693,Ferguson Crest winery US,33.313613,-96.659946,1,OK
8694,8694,Carmel Hills Winery winery US,43.949742,-103.522590,1,OK
8696,8696,Castello Bonomi winery Italy,45.571648,9.967799,1,OK
8697,8697,Barboursville Vineyards winery US,38.159582,-78.277084,1,OK


In [6]:
# Create lists of latitudes and longitudes
lats = list(latlong_df['latitude'])
lngs = list(latlong_df['longitude'])

wineries = list(latlong_df['winery_search_term'])

In [7]:
# Starting URL for Weather Map API Call.
url = "http://history.openweathermap.org/data/2.5/aggregated/year?units=imperial&appid=" + weather_api_key

In [8]:
def get_weather_data (winery, winery_count):
    # Create endpoint URL with each winery.
    winery_url = url + "&lat=" + str(lats[winery_count]) + "&lon=" + str(lngs[winery_count])
    
    winery_temp_data  = []
    winery_pressure_data = []
    winery_humidity_data =[]
    winery_precip_data = []
    
    #set day counter
    i = 0
    
    # Parse the JSON and retrieve data.
    winery_weather = requests.get(winery_url).json()
    
    for result in winery_weather:
        temp = winery_weather["result"][i]["temp"]["mean"]
        winery_temp_data.append(temp)
        pressure = winery_weather["result"][i]["pressure"]["mean"]
        winery_pressure_data.append(pressure)
        humidity = winery_weather["result"][i]["humidity"]["mean"]
        winery_humidity_data.append(humidity)
        precip = winery_weather["result"][i]["precipitation"]["mean"]
        winery_precip_data.append(precip)
        
        i += 1
        
    # calculate averages
    avg_temp = sum(winery_temp_data) / len(winery_temp_data)
    avg_pressure = sum(winery_pressure_data) / len(winery_pressure_data)
    avg_humidity = sum(winery_humidity_data) / len(winery_humidity_data)
    avg_precip = sum(winery_precip_data) / len(winery_precip_data)
    
    return avg_temp, avg_pressure, avg_humidity, avg_precip

In [10]:
# Create an empty list to hold the weather data.
winery_weather_data = []
output_filename = 'winery_weather.csv'

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
winery_count = 0

# Loop through all the wineries in the list.
for i, winery in enumerate(wineries):

    # Group wineries in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {winery}")
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the wineries.
    try:
        # Parse the JSON and retrieve data.
        avg_temp, avg_pressure, avg_humidity, avg_precip = get_weather_data(winery, winery_count)
               
        # Append the winery information into city_data list.
        winery_weather_data.append({"Winery": wineries[winery_count],
                          "Average Temperature (Kelvin)": avg_temp,
                          "Average Air Pressure (hPa)": avg_pressure,
                          "Average Humidity (%)": avg_humidity,
                          "Average Precipitation (mm)": avg_precip})

    # If an error is experienced, skip the winery.
    except:
        print("Weather not found. Skipping...")
        pass
    
    # Add to winery count
    winery_count += 1
    
    # Print status every 100 addresses
    if len(winery_weather_data) % 100 == 0:
    	print("Completed {} of {} address".format(len(winery_weather_data), len(wineries)))
            
    # Every 500 addresses, save progress to file(in case of a failure so we have something!)
    if len(winery_weather_data) % 500 == 0:
        pd.DataFrame(winery_weather_data).to_csv("{}_bak".format(output_filename))

# save full results to CSV
pd.DataFrame(winery_weather_data).to_csv(output_filename)
    


    # Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | St. Julian winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 1 | Sweet Cheeks winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 1 | Kirkland Signature winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 1 | Louis M. Martini winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 1 | Richard Böcking winery Germany
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 1 | Bianchi winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 1 | Castello di Amorosa winery US
----

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 8 of Set 2 | G7 winery Chile
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 9 of Set 2 | Le Cadeau winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 10 of Set 2 | Stoneleigh winery New Zealand
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 11 of Set 2 | Kenwood winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 12 of Set 2 | La Chablisienne winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 13 of Set 2 | Manzoni winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 14 of Set 2 | McIntyre Vineyar

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 15 of Set 3 | Mémoires winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 16 of Set 3 | Viña Tarapacá winery Chile
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 17 of Set 3 | Robert Weil winery Germany
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 18 of Set 3 | Rochioli winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 19 of Set 3 | Louis Latour winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 20 of Set 3 | Château de la Tour winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 21 o

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 22 of Set 4 | Rex Hill winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 23 of Set 4 | Ruby winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 24 of Set 4 | Structure winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 25 of Set 4 | Tumwater winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 26 of Set 4 | Joseph Jewell winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 27 of Set 4 | Le Vigne winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 28 of Set 4 | Maximin Grünhäuser winery Germ

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 29 of Set 5 | Va Piano winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 30 of Set 5 | Y Rousseau winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 31 of Set 5 | Paradise Ridge winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 32 of Set 5 | Poet's Leap winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 33 of Set 5 | Raphael winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 34 of Set 5 | Snoqualmie winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 35 of Set 5 | The Withers Winery w

...

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 47 of Set 155 | Quinta do Vallado winery Portugal
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 48 of Set 155 | Rose & Son winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 49 of Set 155 | Kathy Lynskey winery New Zealand
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 50 of Set 155 | Sirena Dunarii winery Romania
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 1 of Set 156 | Domaine de la Chapelle Saint-Victor winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 156 | Avontuur winery South Africa
-----------------------------
Data Retrieval Complete      
-

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 1 of Set 157 | Turchi winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 2 of Set 157 | Red Mare winery US
Completed 7700 of 7994 address
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 157 | Clos Marguerite winery New Zealand
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 157 | Gallo Estate winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 157 | Pyrenees winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 157 | Point Concepción winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
P

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 158 | Domaine les Fouques winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 158 | Allandale winery Australia
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 8 of Set 158 | Laborie winery South Africa
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 9 of Set 158 | Hopkins Vineyard winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 10 of Set 158 | Greg Norman Estates winery Australia
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 11 of Set 158 | Yaffo winery Israel
-----------------------------
Data Retrieval Complete      
---------------------------

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 11 of Set 159 | Parker winery Australia
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 12 of Set 159 | Le Vigne di Ca' Nova winery Italy
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 13 of Set 159 | Domaine Perzinsky winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 14 of Set 159 | Marchand & Burch winery Australia
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 15 of Set 159 | Château de Landué winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 16 of Set 159 | Maple Ridge winery US
-----------------------------
Data Retrieval Complete      
------------------

-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 16 of Set 160 | Domaine de la Chezatte winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 17 of Set 160 | Villa di Bagnolo winery Italy
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 18 of Set 160 | Waugh Cellars winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 19 of Set 160 | Karma winery US
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 20 of Set 160 | Manoir du Capucin winery France
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 21 of Set 160 | Château de Garros winery France
-----------------------------
Data Retrieval Complete      
-------------------------

In [16]:
# update column name in df and csv
winery_weather_data_df = pd.DataFrame(winery_weather_data)
winery_weather_data_df.rename(columns={'Average Precipitation (mm)': 'Average Daily Precipitation (mm)'}, inplace=True)

winery_weather_data_df.to_csv(output_filename)